In [2]:
import numpy as np
import pandas as pd
import time

In [3]:
def making_array_np():
    df = pd.read_csv('clear.csv', index_col=False, header=0, delimiter=';',
                          names=['Date', 'Time', 'Global_active_power', 'Global_reactive_power', 'Voltage',
                                 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3'])


    arr = df.values
    return arr

In [4]:
def making_df():
    df = pd.read_csv('clear.csv', index_col=False, header=0, delimiter=';',
                    names=['Date', 'Time', 'Global_active_power', 'Global_reactive_power', 'Voltage',
                        'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3'])
    return df

Обрати всі рядки, у яких загальна активна споживана
потужність перевищує 5 кВт.

In [5]:
def moreThan5kWt_np(arr):
    print(arr[arr[:, 2] > 5])   

In [6]:
def moreThan5kWt_df(df):
    print(df[df['Global_active_power']>5])

Обрати всі рядки, у яких вольтаж перевищую 235 В.

In [7]:
def moreThan235Voltage_np(arr):
    print(arr[arr[:, 4] > 235])

In [8]:
def moreThan235Voltage_df(df):    
    print(df[df['Voltage']>235])

Oбрати всі рядки, у яких сила струму лежить в межах
19-20 А, для них виявити ті, у яких пральна машина та холодильних(7) споживають більше, ніж бойлер та кондиціонер(8).

In [9]:
def third_task_np(arr):
    print(arr[(arr[:,5]>19)&(arr[:,5]<20)&(arr[:,7]>arr[:,8])])

In [10]:
def third_task_df(df):
    print(df[(df['Global_intensity']>19) & (df['Global_intensity']<20) & (df['Sub_metering_2']>df['Sub_metering_3'])])

Обрати випадковим чином 500000 рядків (без повторів
елементів вибірки), для них обчислити середні величини усіх 3-х груп споживання електричної енергії.

In [11]:
def fourth_task_np(arr):
    array = arr[np.random.randint(arr.shape[0],size=50000), :]#рандомно выбираем из них 50к
    average = np.array((array[:,5] + array[:,6]  + array[:,7])/3)#считаем среднее для 3 групп, в результате одномерный массив в рядочек
    average = average.reshape((50000,1))
    array = np.concatenate((array, average), axis=1)# добавляем новый столбик
    print(array)

In [12]:
def fourth_task_df(df):
    df = df.sample(500000)#500000 рандомных строчек
    df['Average'] = df[['Sub_metering_1','Sub_metering_2','Sub_metering_3']].mean(axis =1)

Обрати ті рядки, які по часу >= 18-00 споживають понад 6
кВт за хвилину в середньому, серед відібраних визначити ті, у яких основне споживання електроенергії у вказаний проміжок часу припадає на пральну машину, сушарку, холодильник та освітлення (група 2 є найбільшою), а потім обрати кожен третій результат із першої половини та кожен четвертий результат із другої половини. 

In [13]:
def fifth_task_np(arr):
    array = arr[(arr[:, 1] > '18:00:00') & ((arr[:, 2] +  arr[:,3]) > 6) & (arr[:, 6] > arr[:, 7]) &(arr[:, 7] > arr[:, 8])]
    array1 = array[::3]
    array2 = array[::4]
    print(array1)

In [14]:
def fifth_task_df(df):    
    df =df[(df['Time']>'18:00:00')&((df['Global_active_power']+df['Global_reactive_power'])>6)&(df['Sub_metering_2']>df['Sub_metering_1'])&(df['Sub_metering_2']>df['Sub_metering_3'])]
    length = int(len(df.index)/2)
    df1 = df[:length].iloc[::3,:]
    df2 = df[length+1:].iloc[::4,:]
    print(df1)


In [17]:
arr = making_array_np()
df = making_df()

start = time.time()
moreThan5kWt_np(arr)
end = time.time()
diff_np = end-start

start1 = time.time()
moreThan5kWt_df(df)
end1 = time.time()
diff_df = end1-start1

print("For first task with NumPy: ",diff_np)
print("For first task with DataFrame: ",diff_df)

[['16/12/2006' '17:25:00' 5.36 ... 0.0 1.0 16.0]
 ['16/12/2006' '17:26:00' 5.374 ... 0.0 2.0 17.0]
 ['16/12/2006' '17:27:00' 5.388 ... 0.0 1.0 17.0]
 ...
 ['24/11/2010' '07:50:00' 5.172000000000001 ... 0.0 38.0 17.0]
 ['24/11/2010' '07:51:00' 5.75 ... 0.0 39.0 17.0]
 ['25/11/2010' '07:21:00' 5.074 ... 1.0 2.0 18.0]]
               Date      Time  Global_active_power  Global_reactive_power  \
1        16/12/2006  17:25:00                5.360                  0.436   
2        16/12/2006  17:26:00                5.374                  0.498   
3        16/12/2006  17:27:00                5.388                  0.502   
11       16/12/2006  17:35:00                5.412                  0.470   
12       16/12/2006  17:36:00                5.224                  0.478   
...             ...       ...                  ...                    ...   
2043377  22/11/2010  18:40:00                5.408                  0.150   
2043378  22/11/2010  18:41:00                5.528                

In [16]:
start = time.time()
moreThan235Voltage_np(arr)
end = time.time()
diff_np = end-start

start1 = time.time()
moreThan235Voltage_df(df)
end1 = time.time()
diff_df = end1-start1

print("For second task with NumPy: ",diff_np)
print("For second task with DataFrame: ",diff_df)

[['16/12/2006' '17:28:00' 3.6660000000000004 ... 0.0 1.0 17.0]
 ['16/12/2006' '17:29:00' 3.52 ... 0.0 2.0 17.0]
 ['16/12/2006' '17:30:00' 3.702 ... 0.0 1.0 17.0]
 ...
 ['26/11/2010' '21:00:00' 0.938 ... 0.0 0.0 0.0]
 ['26/11/2010' '21:01:00' 0.934 ... 0.0 0.0 0.0]
 ['26/11/2010' '21:02:00' 0.932 ... 0.0 0.0 0.0]]
               Date      Time  Global_active_power  Global_reactive_power  \
4        16/12/2006  17:28:00                3.666                  0.528   
5        16/12/2006  17:29:00                3.520                  0.522   
6        16/12/2006  17:30:00                3.702                  0.520   
7        16/12/2006  17:31:00                3.700                  0.520   
14       16/12/2006  17:38:00                4.054                  0.422   
...             ...       ...                  ...                    ...   
2049275  26/11/2010  20:58:00                0.946                  0.000   
2049276  26/11/2010  20:59:00                0.944                  0

In [104]:
start = time.time()
third_task_np(arr)
end = time.time()
diff_np = end -start

start = time.time()
third_task_df(df)
end = time.time()
diff_df = end -start

print("For third task with NumPy: ", diff_np)
print("For third task with DataFrame: ", diff_df)

[['17/12/2006' '01:04:00' 4.582 ... 0.0 13.0 0.0]
 ['17/12/2006' '01:08:00' 4.618 ... 0.0 27.0 0.0]
 ['17/12/2006' '01:19:00' 4.636 ... 0.0 36.0 0.0]
 ...
 ['24/11/2010' '07:54:00' 4.694 ... 0.0 39.0 18.0]
 ['24/11/2010' '07:55:00' 4.602 ... 0.0 40.0 17.0]
 ['24/11/2010' '07:57:00' 4.626 ... 0.0 39.0 17.0]]
               Date      Time  Global_active_power  Global_reactive_power  \
460      17/12/2006  01:04:00                4.582                  0.258   
464      17/12/2006  01:08:00                4.618                  0.104   
475      17/12/2006  01:19:00                4.636                  0.140   
476      17/12/2006  01:20:00                4.634                  0.152   
477      17/12/2006  01:21:00                4.652                  0.142   
...             ...       ...                  ...                    ...   
2045609  24/11/2010  07:52:00                4.636                  0.000   
2045610  24/11/2010  07:53:00                4.666                  0.000  

In [105]:
start = time.time()
fourth_task_np(arr)
end = time.time()
diff_np = end -start

start = time.time()
fourth_task_df(df)
end = time.time()
diff_df = end -start

print("For fourth task with NumPy: ", diff_np)
print("For fourth task with DataFrame: ", diff_df)

[['9/12/2009' '05:18:00' 0.40399999999999997 ... 1.0 1.0
  0.9333333333333332]
 ['2/12/2007' '16:12:00' 3.9160000000000004 ... 25.0 18.0
  13.733333333333334]
 ['23/8/2009' '03:39:00' 0.38799999999999996 ... 1.0 1.0
  1.0666666666666667]
 ...
 ['11/11/2010' '22:56:00' 0.386 ... 1.0 0.0 0.8666666666666667]
 ['3/2/2009' '20:48:00' 2.0540000000000003 ... 0.0 0.0 2.8000000000000003]
 ['23/7/2009' '04:52:00' 0.204 ... 1.0 0.0 0.6666666666666666]]
For fourth task with NumPy:  0.16156649589538574
For fourth task with DataFrame:  0.2742645740509033


In [106]:
start = time.time()
fifth_task_np(arr)
end = time.time()
diff_np = end -start

start = time.time()
fifth_task_np(arr)
end = time.time()
diff_df =end -start

print("For fifth task with NumPy: ", diff_np)
print("For fifth task with DataFrame: ", diff_df)

[['23/12/2006' '22:37:00' 8.632 ... 38.0 1.0 0.0]
 ['23/12/2006' '22:49:00' 5.892 ... 36.0 2.0 0.0]
 ['25/12/2006' '21:49:00' 6.362 ... 37.0 32.0 17.0]
 ...
 ['21/6/2010' '18:57:00' 7.022 ... 67.0 24.0 18.0]
 ['11/10/2010' '21:39:00' 5.938 ... 37.0 32.0 18.0]
 ['4/11/2010' '19:16:00' 6.696000000000001 ... 40.0 35.0 0.0]]
[['23/12/2006' '22:37:00' 8.632 ... 38.0 1.0 0.0]
 ['23/12/2006' '22:49:00' 5.892 ... 36.0 2.0 0.0]
 ['25/12/2006' '21:49:00' 6.362 ... 37.0 32.0 17.0]
 ...
 ['21/6/2010' '18:57:00' 7.022 ... 67.0 24.0 18.0]
 ['11/10/2010' '21:39:00' 5.938 ... 37.0 32.0 18.0]
 ['4/11/2010' '19:16:00' 6.696000000000001 ... 40.0 35.0 0.0]]
For fifth task with NumPy:  0.3959383964538574
For fifth task with DataFrame:  0.350067138671875
